In [ ]:
from shapely.geometry import Polygon, MultiPolygon, Point
from shapely.ops import unary_union
from owslib.wfs import WebFeatureService
import pandas as pd
import geopandas as gpd
import logging
import matplotlib.pyplot as plt
import leafmap

In [ ]:
logger = logging.getLogger(__name__)

GRUPOS_CAMADAS = {
    #  "valor_patrimonio_ha": [
    #     "geonode:car2022_municipios2019_dimensaopatrimonio_valorha2020_wgs84utf"
    # ]
    "limites_territoriais": [
        "GeoJundiai:1979_IGC_limite_municipio",
        "GeoJundiai:L_8683-2016_m04_perimetro-urbano-rural",
        "GeoJundiai:L_9321-2019_m02-Limite_corredor_desenv_regional",
        "GeoJundiai:LC_461-2008_regioes",
        "GeoJundiai:LC_461-2008_bairros",
        "GeoJundiai:bairros_em_1968"
    ],
    "areas_verdes": [
        "GeoJundiai:LC_417-2004_SistProtSerradoJapi",
        "GeoJundiai:L_8683-2016_m03_rem-veg-nat",
        "GeoJundiai:L_9321-2019_m04-Remanescente_Vegetacao-Cerrado",
        "GeoJundiai:L_9321-2019_m04-Remanescente_Vegetacao-Mata_Atlantica",
        "GeoJundiai:L_8683-2016_m13_parques-municipais",
        "GeoJundiai:L_4095-1984_APAJundiai"
    ],
    "hidrografia": [
        "GeoJundiai:L_2405-1980_Manancial",
        "GeoJundiai:L_8683-2016_hidrografia-principal",
        "GeoJundiai:L_8683-2016_m01_bacias-hidrograficas",
        "GeoJundiai:L_8683-2016_m01_recarga-hidrica",
        "GeoJundiai:L_8683-2016_m01_represas",
        "GeoJundiai:L_8683-2016_m02_nascentes",
        "GeoJundiai:L_9321-2019_m01-Bacias_Hidrograficas",
        "GeoJundiai:hidrografia_aero_1993"
    ],
    "zoneamento": [
        "GeoJundiai:L_10177-2024_m01_Macrozoneamento", #erro#
        "GeoJundiai:L_10177-2024_m02_Zoneamento", #erro#
        "GeoJundiai:L_7857-2012_perimetro-urbano",
        "GeoJundiai:L_7858-2012-zoneamento",
        "GeoJundiai:L_8683-2016_m05_macrozoneamento",
        "GeoJundiai:L_8683-2016_m06_zoneamento",
        "GeoJundiai:L_9321-2019_m01-Macrozoneamento",
        "GeoJundiai:L_9321-2019_m02-Zoneamento",
        "GeoJundiai:L_9806-2022_m02_zoneamento",
        "GeoJundiai:v_macrozoneamento",
        "GeoJundiai:v_usodosolo"
    ],
    "infraestrutura_urbana": [
        "GeoJundiai:L_8683-2016_m12_vias_func_urbanistica",
        "GeoJundiai:L_8683-2016_m13_redecicloviaria",
        "GeoJundiai:L_9321-2019_m08-Rede_Cicloviaria",
        "GeoJundiai:L_8683-2016_m13_terminais-situ",
        "GeoJundiai:onibus_terminais",
        "GeoJundiai:v_classdiretrizviar",
        "GeoJundiai:v_classviaria_atual",
        "GeoJundiai:v_diretrizes_viarias"
    ],
    "patrimonio_cultural": [
        "GeoJundiai:L_8683-2016_m09_ZEIHC-bens-tombados",
        "GeoJundiai:L_9321-2019_m03-Bens_Tombados",
        "GeoJundiai:m03_bens_tombados_cica",
        "GeoJundiai:L_8683-2016_m09_zeihc",
        "GeoJundiai:L_9321-2019_m03-ZEIHC"
    ],
    "areas_especiais": [
        "GeoJundiai:L_8683-2016_m07_zeis",
        "GeoJundiai:L_8683-2016_m08_zerfie",
        "GeoJundiai:L_10177-2024_m06_ZEIS",
        "GeoJundiai:L_8683-2016_m10_zepam",
        "GeoJundiai:v_zeis",
        "GeoJundiai:v_zerf"
    ],
    "cadastro_fundiario": [
        "GeoJundiai:L_10177-2024_m05_Cadastro_Fundiário",
        "GeoJundiai:L_9321-2019_m05-Cadastro_Fundiario",
        "GeoJundiai:edificacoes",
        "GeoJundiai:v_loteamentos"
    ],
    "enderecamento": [
        "GeoJundiai:enderecamento_cep",
        "GeoJundiai:L_9321-2019_m02-Logradouros",
        "GeoJundiai:v_denominacoes",
        "GeoJundiai:v_logr_nome",
        "GeoJundiai:denominacoes"
    ]
}

In [ ]:
from backend.app.models.esquema import RequisicaoAnalise 
from backend.app.services.geocodificacao import geocodificar_endereco
from backend.app.services.espacial import calcular_area_verde

def analisar_area(endereco:str):
    try:
        coordenadas = geocodificar_endereco(endereco)
        print(coordenadas)
        longitude, latitude = coordenadas

        ponto = gpd.GeoSeries([Point(longitude, latitude)], crs="EPSG:4326")
        ponto = ponto.to_crs("EPSG:31983")

        buffer = ponto.buffer(500)

        buffer_gdf = gpd.GeoDataFrame(geometry=buffer, crs="EPSG:31983")
        return buffer_gdf
    except Exception as e:
        print(e)

In [ ]:
def carregar_camadas(grupo: str, combinar: bool = True) -> gpd.GeoDataFrame | dict:
    print(f'camada solicitada> {grupo}')
    print('chegou no carregar camadas')
    """
    Carrega camadas de um grupo específico do WFS

    Args:
        grupo: Nome do grupo de camadas
        combinar: Se True, combina todas as camadas em um único GeoDataFrame

    Returns:
        GeoDataFrame combinado ou dicionário de GeoDataFrames por camada
    """
    if grupo not in GRUPOS_CAMADAS:
        raise ValueError(f"Grupo '{grupo}' não existe. Grupos válidos: {list(GRUPOS_CAMADAS.keys())}")

    url_wfs = "https://geo.jundiai.sp.gov.br/geoserver/ows?service=WFS&acceptversions=2.0.0&request=GetCapabilities"
    camadas = GRUPOS_CAMADAS[grupo]

    if combinar:
        gdf_combinado = gpd.GeoDataFrame()
    else:
        resultado = {}

    for camada in camadas:
        try:
            wfs = WebFeatureService(url=url_wfs, version='2.0.0')
            response = wfs.getfeature(
                typename=camada,
                outputFormat='application/json',
                srsname='EPSG:31983'
            )
            gdf = gpd.read_file(response)

            if gdf.crs is None:
                gdf.crs = "EPSG:31983"

            gdf = gdf[gdf.geometry.notnull()]
            gdf = gdf[gdf.geometry.apply(lambda geom: isinstance(geom, (Polygon, MultiPolygon)))]

            if combinar:
                gdf_combinado = pd.concat([gdf_combinado, gdf], ignore_index=True)
            else:
                resultado[camada] = gdf

        except Exception as e:
            logger.error(f"Erro ao carregar camada {camada}: {str(e)}")

    if combinar:
        return gdf_combinado
    else:
        return resultado


In [ ]:
def carregar_areas_verdes() -> gpd.GeoDataFrame:
    """
    Função específica para carregar camadas de áreas verdes (combinadas),
    com opção de dissolver todas as geometrias.

    Args:
        dissolver: Se True, aplica .dissolve() no GeoDataFrame retornado

    Returns:
        GeoDataFrame com áreas verdes
    """
    gdf = carregar_camadas("areas_verdes")

    return gdf

In [ ]:
import geopandas as gpd
from shapely.geometry import Point
import leafmap

gdf_buffer = analisar_area("Avenida Humberto Cereser")
gdf_buffer["geometry"] = gdf_buffer.buffer(0)

gdf_areas_verdes = carregar_areas_verdes()
gdf_areas_verdes = gdf_areas_verdes[gdf_areas_verdes.is_valid]
gdf_areas_verdes["geometry"] = gdf_areas_verdes.buffer(0)

# print(gdf_areas_verdes.geometry.area.sum())

# display(gdf_buffer)

# display(gdf_areas_verdes)

intersecao = gpd.overlay(gdf_areas_verdes, gdf_buffer, how='intersection')

# display(intersecao)

area_buffer = gdf_buffer.geometry.area.iloc[0]

area_intersecao = (intersecao.geometry.area.sum()) - area_buffer
porcentagem = (area_intersecao / area_buffer) * 100

print(f"Área do buffer (m²): {area_buffer:.2f}")
print(f"Área verde dentro do buffer (m²): {area_intersecao:.2f}")
print(f"Porcentagem de área verde no entorno: {porcentagem:.2f}%")

In [ ]:
gdf_buffer.explore()

In [ ]:
gdf_areas_verdes.explore()

In [ ]:
intersecao.explore()

In [ ]:
print(intersecao.geometry.iloc[0])


In [ ]:
print(gdf_buffer.geometry.iloc[0])

In [ ]:
m = leafmap.Map(center=[-23.2, -46.9], zoom=14)

m.add_gdf(gdf_buffer, layer_name='Buffer 500m', style={"fillColor": "blue", "fillOpacity": 0.2})
m.add_gdf(gdf_areas_verdes, layer_name='Áreas Verdes', style={"fillColor": "green", "fillOpacity": 0.4})
m.add_gdf(intersecao, layer_name='Interseção', style={"fillColor": "red", "fillOpacity": 0.6})
m.add_gdf(gdf_buffer, layer_name='Endereço', style={"color": "black", "radius": 5})

m.add_legend(title="Camadas", labels=["Buffer", "Áreas Verdes", "Interseção"], colors=["blue", "green", "red"])
m